# QUANTITATIVE RISK MANAGEMENT

### <font color='MediumVioletRed' style="font-size:20px"><b>Installing and importing Libraries:</b></font>

In [ ]:
# Installing additional libraries
!pip install yfinance
!pip install yahoofinancials

In [3]:
# Standard library
from typing import List

# Third-party libraries
import numpy as np
import pandas as pd
import matplotlib. pyplot as plt
from scipy.stats import norm
import scipy
import yfinance as yf
from yahoofinancials import YahooFinancials
from tabulate import tabulate
import plotly.graph_objects as go

Construct a portfolio of 10 S&P500 stocks with equal weights. Make a short python example that shows the development of Value at Risk and Expected Shortfall (ES) on a graph for the last 150 days. Compare it with VaR and ES of S&P500 for the same time frame.

In [4]:
def get_portfolio_asset_returns(stocks:List[str], start:str, end:str) -> pd.DataFrame:
  """Gets portfolio asset daily returns for a give period"""
  stock_data = yf.download(stocks, start, end, progress=False)
  stock_data = stock_data['Close']
  returns = stock_data.pct_change()
  return returns

Let's construct a equally-weighted portfolio of 10 stocks:
- Netflix
- Draft Kings
- Delta Airlines
- Enbridge Inc
- Boeing
- AT&T
- The Walt Disney company
- AMZN
- RBC
- Costco

In [5]:
# Portfolio assets
stocks = ["NFLX", "DKNG", "DAL", "ENB.TO", "BA", "T", "DIS", "AMZN", "RY.TO", "COST"]

# Weights for equally weighted portfolio
weights = np.array([1/10]*10)

In [24]:
# Time horizon
start = '2020-01-01'
end = '2023-09-01'

# Stock returns
returns = get_portfolio_asset_returns(stocks=stocks, start=start, end=end)
returns.tail()

,AMZN,BA,COST,DAL,DIS,DKNG,ENB.TO,NFLX,RY.TO,T
Date,,,,,,,,,,
2023-08-25,0.010771,0.028071,0.004326,-0.004773,0.010792,0.033028,0.008832,0.022363,-0.013048,-0.002827
2023-08-28,-0.000900,0.016338,0.004082,0.017026,0.009597,-0.001421,0.007474,0.004879,0.008263,0.007087
2023-08-29,0.013294,0.000837,0.012272,0.032068,0.002852,0.027748,0.007630,0.028537,0.005245,0.039409
2023-08-30,0.001186,0.007041,-0.000940,-0.012337,-0.001422,0.026999,0.003997,0.010884,0.002609,-0.002708
2023-08-31,0.021766,-0.021062,0.012946,-0.008096,-0.007119,-0.000674,-0.006076,-0.002278,-0.010083,0.004073


In [7]:
# S&P500 returns
snp500_returns = get_portfolio_asset_returns(stocks=["^GSPC"], start=start, end=end)
snp500_returns.tail()

Date
2023-08-25    0.006718
2023-08-28    0.006265
2023-08-29    0.014508
2023-08-30    0.003833
2023-08-31   -0.001597
Name: Close, dtype: float64

In [8]:
# Plotting stock daily returns against S&P500 returns
fig=go.Figure()
for ticker in stocks:
  fig.add_trace(go.Scatter(x=returns.index, y=returns[ticker], name=ticker))
fig.add_trace(go.Scatter(x=snp500_returns.index, y=snp500_returns.values, name="S&P 500"))
fig.update_layout(xaxis_title='Dates',yaxis_title='Daily returns',
        title=dict(text='Stocks daily returns', x=0.5, y=0.87, font=dict(size=22,color='Navy')))
fig.show()

Let's calculate VAR and ES for both the S&P500 and our portfolio and compare the results.

1. Calulating VAR and ES for the S&P 500 index

In [9]:
def var_historical(returns, confidence_level=0.05):
  """Calculates historical VAR with a given conficence level, non-parametric metric"""

  return returns.quantile(confidence_level, interpolation='linear')

In [10]:
def es_historical(returns, confidence_level=0.05):
  """Calculates Expected Shortfall (ES)"""

  # Calculating VaR
  var = var_historical(returns, confidence_level)

	# Given the VAR, ES (AVAR, CVAR) is the average of the worst losses
  return returns[returns.lt(var)].mean()

In [12]:
snp500_returns.describe()

count    922.000000
mean       0.000465
std        0.014963
min       -0.119841
25%       -0.006304
50%        0.000794
75%        0.007964
max        0.093828
Name: Close, dtype: float64

In [13]:
# Calculating 95% VAR and 95% ES for the last 150 days
snp500_var_95 = []
snp500_var_99 = []
snp500_es_95 = []
snp500_es_99 = []

for i in range(len(snp500_returns) -150, len(snp500_returns)):
  # Calculating VAR
  snp500_var_95.append(-var_historical(snp500_returns[0:i].dropna(), confidence_level=0.05))
  snp500_var_99.append(-var_historical(snp500_returns[0:i].dropna(), confidence_level=0.01))

  # Calcularing ES
  snp500_es_95.append(-es_historical(snp500_returns[0:i].dropna(), confidence_level=0.05))
  snp500_es_99.append(-es_historical(snp500_returns[0:i].dropna(), confidence_level=0.01))

In [14]:
# Plotting VAR and AVAR for S&P500 index
fig=go.Figure()
fig.add_trace(go.Scatter(x=list(range(150)), y=snp500_var_95, name="95% VAR"))
fig.add_trace(go.Scatter(x=list(range(150)), y=snp500_var_99, name="99% VAR"))
fig.add_trace(go.Scatter(x=list(range(150)), y=snp500_es_95, name="95% ES"))
fig.add_trace(go.Scatter(x=list(range(150)), y=snp500_es_99, name="99% ES"))
fig.add_trace(go.Scatter(x=list(range(150)), y=snp500_returns[len(snp500_returns) -150: len(snp500_returns)], name="S&P500 daily returns"))
fig.update_layout(xaxis_title='Last 150 days',yaxis_title='Expected loss for the given risk measure',
        title=dict(text='S&P500 VAR and ES', x=0.5, y=0.87, font=dict(size=22,color='Navy')))
fig.show()

On the plot above we can see the 95% and 99% VAR and ES for S&P500 index.

We can see that 99% VAR/ES is higher than 95% VAR/ES for abound 2 basis points on average.

What is also expected from theoretical properties and here we can observe, is that ES risk measure give us greater values, because it calculates the average of the VaRs which are larger than the VaR at tail probability  𝜖.

2. Calulating VAR and ES for our portfolio

In [15]:
# Portfolio asset statistical properties
returns.describe()

,AMZN,BA,COST,DAL,DIS,DKNG,ENB.TO,NFLX,RY.TO,T
count,940.000000,940.000000,940.000000,940.000000,940.000000,940.000000,940.000000,940.000000,940.000000,940.000000
mean,0.000684,0.000198,0.000803,0.000228,-0.000348,0.002195,0.000072,0.000791,0.000275,-0.000571
std,0.023901,0.035220,0.016078,0.033550,0.022806,0.046907,0.017847,0.030928,0.014379,0.017763
min,-0.140494,-0.238484,-0.124513,-0.259924,-0.131632,-0.278239,-0.165037,-0.351166,-0.105383,-0.104061
25%,-0.012144,-0.015189,-0.006799,-0.015254,-0.011319,-0.025958,-0.005824,-0.014114,-0.004550,-0.008188
50%,0.000315,-0.000210,0.000678,-0.000108,-0.000499,0.000000,0.000981,0.000072,0.000830,0.000000
75%,0.012923,0.014345,0.008596,0.016128,0.010049,0.028344,0.006799,0.014529,0.005984,0.006791
max,0.135359,0.243186,0.099595,0.210171,0.144123,0.172697,0.206858,0.168543,0.148963,0.100223


In [16]:
# Portfolio asset returns
returns.tail()

,AMZN,BA,COST,DAL,DIS,DKNG,ENB.TO,NFLX,RY.TO,T
Date,,,,,,,,,,
2023-08-25,0.010771,0.028071,0.004326,-0.004773,0.010792,0.033028,0.008832,0.022363,-0.013048,-0.002827
2023-08-28,-0.000900,0.016338,0.004082,0.017026,0.009597,-0.001421,0.007474,0.004879,0.008263,0.007087
2023-08-29,0.013294,0.000837,0.012272,0.032068,0.002852,0.027748,0.007630,0.028537,0.005245,0.039409
2023-08-30,0.001186,0.007041,-0.000940,-0.012337,-0.001422,0.026999,0.003997,0.010884,0.002609,-0.002708
2023-08-31,0.021766,-0.021062,0.012946,-0.008096,-0.007119,-0.000674,-0.006076,-0.002278,-0.010083,0.004073


In [17]:
# Portfolio returns
returns_p = returns * weights
returns_p = returns_p.dropna().sum(axis=1)
returns_p

Date
2020-01-03   -0.005261
2020-01-06    0.005533
2020-01-07    0.000491
2020-01-08    0.008271
2020-01-09    0.010182
                ...   
2023-08-25    0.009753
2023-08-28    0.007243
2023-08-29    0.016989
2023-08-30    0.003531
2023-08-31   -0.001660
Length: 940, dtype: float64

In [18]:
# Plotting portfolio daily returns against S&P500 returns
fig=go.Figure()
fig.add_trace(go.Scatter(x=snp500_returns.index, y=snp500_returns.values, name="S&P 500"))
fig.add_trace(go.Scatter(x=returns_p.index, y=returns_p.values, name="Portfolio"))
fig.update_layout(xaxis_title='Dates',yaxis_title='Daily returns',
        title=dict(text='Portfolio vs S&P 500 daily returns', x=0.5, y=0.87, font=dict(size=22,color='Navy')))
fig.show()

In [19]:
# Calculating 95% VAR and 95% ES for the last 150 days
portfolio_var_95 = []
portfolio_var_99 = []
portfolio_es_95 = []
portfolio_es_99 = []

for i in range(len(returns_p) -150, len(returns_p)):
  # Calculating VAR
  portfolio_var_95.append(-var_historical(returns_p[0:i].dropna(), confidence_level=0.05))
  portfolio_var_99.append(-var_historical(returns_p[0:i].dropna(), confidence_level=0.01))

  # Calcularing ES
  portfolio_es_95.append(-es_historical(returns_p[0:i].dropna(), confidence_level=0.05))
  portfolio_es_99.append(-es_historical(returns_p[0:i].dropna(), confidence_level=0.01))

In [20]:
# Plotting VAR and AVAR for our portfolio
fig=go.Figure()
fig.add_trace(go.Scatter(x=list(range(150)), y=portfolio_var_95, name="95% VAR"))
fig.add_trace(go.Scatter(x=list(range(150)), y=portfolio_var_99, name="99% VAR"))
fig.add_trace(go.Scatter(x=list(range(150)), y=portfolio_es_95, name="95% ES"))
fig.add_trace(go.Scatter(x=list(range(150)), y=portfolio_es_99, name="99% ES"))
fig.update_layout(xaxis_title='Last 150 days',yaxis_title='Expected loss for the given risk measure',
        title=dict(text='Portfolio VAR and ES', x=0.5, y=0.87, font=dict(size=22,color='Navy')))
fig.show()

Similar like for the VAR and ES for the S&P500, we can observe that for our portfolio over last 150 days maximum expected loss for the 95% and 99% VAR and ES was slightly decreasing.

3. VAR and ES comparison between our portfolio and S&P 500 index

Now, lets compare the risk measures for our portfolio and for the S&P 500 index. As we already hand picked 10 stocks from the list of S&P 500 companies, given that we have only 10 stocks compared to the 500 stocks in the index, we expect to see higher potential losses for our portfolio under the VAR and ES risk measures.

In [23]:
# VAR for our portfolio and S&P 500
fig=go.Figure()
fig.add_trace(go.Scatter(x=list(range(150)), y=portfolio_var_95, name="95% VAR: portfolio"))
fig.add_trace(go.Scatter(x=list(range(150)), y=portfolio_var_99, name="99% VAR: portfolio"))
fig.add_trace(go.Scatter(x=list(range(150)), y=snp500_var_95, name="95% VAR: S&P500"))
fig.add_trace(go.Scatter(x=list(range(150)), y=snp500_var_99, name="99% VAR: S&P500"))
fig.update_layout(xaxis_title='Last 150 days',yaxis_title='Expected loss for the given risk measure',
        title=dict(text='95%/99% VAR: S&P500 vs. our portfolio', x=0.5, y=0.87, font=dict(size=22,color='Navy')))
fig.show()

In [22]:
# ES for our portfolio and S&P 500
fig=go.Figure()
fig.add_trace(go.Scatter(x=list(range(150)), y=portfolio_es_95, name="95% ES: portfolio"))
fig.add_trace(go.Scatter(x=list(range(150)), y=portfolio_es_99, name="99% ES: portfolio"))
fig.add_trace(go.Scatter(x=list(range(150)), y=snp500_es_95, name="95% ES: S&P500"))
fig.add_trace(go.Scatter(x=list(range(150)), y=snp500_es_99, name="99% ES: S&P500"))
fig.update_layout(xaxis_title='Last 150 days',yaxis_title='Expected loss for the given risk measure',
        title=dict(text='95%/99% ES: S&P500 vs. our portfolio', x=0.5, y=0.87, font=dict(size=22,color='Navy')))
fig.show()

As we expected, because of the lower level of diversification of our portfolio compared to the index, we can see that our portfolio is riskier.

For both the 95% and 99% VAR, our maximum expected loss for the portfolio is 0.6 basis points on average greater.

Similar is for  95% and 99% ES, our maximum expected loss for the portfolio is on average greater for 1 basis points.